# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [61]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [67]:
df['delinq_2yrs'].unique()

array([ 0.,  1.,  2.,  4.,  6.,  3.,  7.,  8.,  5.,  9., 10., 13., 11.,
       18., 15., 14., 12., 17.])

In [72]:
for column in columns:
    print(f"{[column]}: {df[column].unique()}")

['loan_amnt']: [10500. 25000. 20000. ... 17850. 16575. 38675.]
['int_rate']: [0.1719 0.2    0.164  0.1474 0.1797 0.1557 0.0881 0.0819 0.1171 0.1033
 0.225  0.139  0.0756 0.0646 0.0702 0.1308 0.124  0.1102 0.25   0.288
 0.29   0.1131 0.1894 0.234  0.1356 0.1298 0.1447 0.2631 0.2235 0.1502
 0.118  0.1691 0.1072 0.1614 0.2935 0.289  0.2892 0.2897 0.1992 0.2534
 0.2089 0.2727 0.2895 0.2437 0.2969 0.3017 0.06   0.3065 0.2872 0.3084
 0.3075 0.3079]
['installment']: [ 375.35  929.09  529.88 ...  254.82 1215.33  226.92]
['home_ownership']: ['RENT' 'MORTGAGE' 'OWN' 'ANY']
['annual_inc']: [ 66000. 105000.  56000. ...  46572.  75005. 520000.]
['verification_status']: ['Source Verified' 'Verified' 'Not Verified']
['issue_d']: ['Mar-2019' 'Feb-2019' 'Jan-2019']
['loan_status']: ['low_risk' 'high_risk']
['pymnt_plan']: ['n']
['dti']: [ 27.24  20.23  24.26 ...  46.01  48.16 147.1 ]
['delinq_2yrs']: [ 0.  1.  2.  4.  6.  3.  7.  8.  5.  9. 10. 13. 11. 18. 15. 14. 12. 17.]
['inq_last_6mths']: [0. 1. 2.

# Split the Data into Training and Testing

In [74]:
#Refer to module 18.7.2
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Create our features
df2=df.copy()


#Use binary encoding on pymnt plan/hardship flag/debt settlement flag = ONE COLUMN EACH
loans_binary_encoding = pd.get_dummies(df2, columns=["initial_list_status", "application_type", "pymnt_plan", "hardship_flag", "debt_settlement_flag"])
loans_binary_encoding.head()

#Use label encoding on:
#home_ownership = OWN/RENT/MORTGAGE/ANY
#verification_status = NOT VERIFIED/SOURCE VERIFIED/VERIFIED
#loan status = HIGH RISK/LOW RISK
#**Loan status is label encoded so it is easier to drop a singular column
df3=loans_binary_encoding.copy()
df3["loan_status"] = le.fit_transform(df3["loan_status"])
df3["verification_status"] = le.fit_transform(df3["verification_status"])
df3["home_ownership"]=le.fit_transform(df3["home_ownership"])

#Use custom encoding on month issue column
#Create a dictionary of months in dataframe
issue_month = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5,
}
#Apply custom function to convert string to integers
df3["month_issued"] = df3["issue_d"].apply(lambda x: issue_month[x])
df3["next_payment"] = df3["next_pymnt_d"].apply(lambda x: issue_month[x])
X = df3.drop(["loan_status","issue_d","next_pymnt_d"], axis = 1)


# Create our target
y = df3["loan_status"]

X.head(15)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_il_high_credit_limit,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,month_issued,next_payment
0,10500.0,0.1719,375.35,3,66000.0,1,27.24,0.0,0.0,8.0,...,61987.0,0,1,1,0,1,1,1,3,5
1,25000.0,0.2000,929.09,1,105000.0,2,20.23,0.0,0.0,17.0,...,49197.0,0,1,1,0,1,1,1,3,5
2,20000.0,0.2000,529.88,1,56000.0,2,24.26,0.0,0.0,8.0,...,43144.0,0,1,1,0,1,1,1,3,5
3,10000.0,0.1640,353.55,3,92000.0,2,31.44,0.0,1.0,10.0,...,76506.0,0,1,1,0,1,1,1,3,5
4,22000.0,0.1474,520.39,1,52000.0,0,18.76,0.0,1.0,14.0,...,20000.0,0,1,1,0,1,1,1,3,5
5,25000.0,0.1797,634.43,1,90000.0,1,18.49,1.0,1.0,14.0,...,369051.0,0,1,1,0,1,1,1,3,5
6,20400.0,0.2000,540.48,3,51315.0,1,25.63,0.0,2.0,8.0,...,27523.0,0,1,1,0,1,1,1,3,5
7,10000.0,0.1557,349.46,2,66000.0,1,9.00,0.0,0.0,11.0,...,12500.0,0,1,1,0,1,1,1,3,5
8,14000.0,0.0881,443.96,3,45000.0,0,10.24,0.0,1.0,9.0,...,3000.0,1,0,1,0,1,1,1,3,5
9,10000.0,0.1474,345.39,3,63295.0,2,39.34,0.0,0.0,27.0,...,80722.0,0,1,1,0,1,1,1,3,5


In [75]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_il_high_credit_limit,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,month_issued,next_payment
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,21.778153,0.217766,0.497697,12.587340,...,5.572240e+04,0.123879,0.876121,0.860340,0.139660,1.0,1.0,1.0,1.726172,4.616839
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,20.199244,0.718367,0.758122,6.022869,...,5.095845e+04,0.329446,0.329446,0.346637,0.346637,0.0,0.0,0.0,0.743862,0.486161
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,2.000000,...,1.270000e+02,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.000000,4.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,13.890000,0.000000,0.000000,8.000000,...,2.288000e+04,0.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0,1.000000,4.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,19.760000,0.000000,0.000000,11.000000,...,4.200000e+04,0.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0,2.000000,5.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,26.660000,0.000000,1.000000,16.000000,...,7.249900e+04,0.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0,2.000000,5.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,999.000000,18.000000,5.000000,72.000000,...,1.426964e+06,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,3.000000,5.000000


In [76]:
# Check the balance of our target values

y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)
X_train.shape

(51612, 87)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [78]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [79]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [82]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,y_pred)

0.6452695488918703

In [81]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   52,    35],
       [ 5258, 11860]], dtype=int64)

In [83]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.60      0.69      0.02      0.64      0.41        87
          1       1.00      0.69      0.60      0.82      0.64      0.42     17118

avg / total       0.99      0.69      0.60      0.81      0.64      0.42     17205



### SMOTE Oversampling

In [84]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
X_train, y_train)

In [85]:
# Train the Logistic Regression model using the resampled data
model=LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [86]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.6308624517043967

In [87]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[   53,    34],
       [ 5948, 11170]], dtype=int64)

In [88]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.61      0.65      0.02      0.63      0.40        87
          1       1.00      0.65      0.61      0.79      0.63      0.40     17118

avg / total       0.99      0.65      0.61      0.78      0.63      0.40     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [89]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 260, 1: 260})

In [90]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [92]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5800528582536633

In [91]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  51,   36],
       [7294, 9824]], dtype=int64)

In [93]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.57      0.01      0.58      0.34        87
          1       1.00      0.57      0.59      0.73      0.58      0.34     17118

avg / total       0.99      0.57      0.59      0.72      0.58      0.34     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [94]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X,y)

In [95]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [96]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5800528582536633

In [97]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  64,   23],
       [7346, 9772]], dtype=int64)

In [98]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.74      0.57      0.02      0.65      0.43        87
          1       1.00      0.57      0.74      0.73      0.65      0.41     17118

avg / total       0.99      0.57      0.73      0.72      0.65      0.41     17205

